## Scrap of town's climate from january 2020 to december 2021.
temp min à 7h, temp max à 19h

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime
import time
import itertools as it
import random

import bs4
from bs4 import BeautifulSoup

from scipy.ndimage.interpolation import shift


import warnings
warnings.filterwarnings('ignore')

#!pip install fake-useragent
#from fake_useragent import UserAgent

### Format of the website

In [2]:
Region = "Occitanie" # "Provence-alpes-c-te-d-azur"  "Rhone-Alpes" #"Ile-de-France" "Aquitaine"   "Provence-alpes-c-te-d-azur" "Occitanie" 
ville =  "Toulouse" #"Marseille" #"lyon" #"Bordeaux" #    "Paris" #

#url = "https://www.historique-meteo.net/france/ile-de-france/paris/"
#url= "https://www.historique-meteo.net/france/aquitaine/bordeaux/"
url = "https://www.historique-meteo.net/france/midi-toulousain/toulouse/"
#url ="https://www.historique-meteo.net/france/lyonnais/lyon/"
#url = "https://www.historique-meteo.net/france/provence-alpes-c-te-d-azur/marseille/"

### Range date of data we want to get

In [3]:
date_debut = '2022/04/15'
date_fin ='2022/04/30'
#date_fin ='2020/01/05' # 5 urls pour tester la fonction

### Getting all the url from date_debut to date_fin

In [4]:
def get_pages(url,deb,fin):
    
    # getting the date range
    def get_rangeDates(deb,fin):
        dateTime = pd.date_range(start=deb, end=fin)
        date = []
        for i in dateTime:
            i = i.date() # getting the date without the time
            i = str(i) # converting to str to use the replace function
            date.append(i)
        return(date)
    pages =[]
    dates = get_rangeDates(deb,fin) 
    for i in dates:
        i = i.replace('-','/') # To have the same date format of the website
        j = url + str(i)
        pages.append(j)
    return(pages)

In [5]:
pages = get_pages(url,date_debut,date_fin)
print(len(pages))
print(pages[-1])

16
https://www.historique-meteo.net/france/midi-toulousain/toulouse/2022/04/30


### Getting the data

In [6]:
# fUNCTIONC TO GET ALL DATAS 
def get_all_data(pages):
    
    # creating the get_data function for one url
    def get_data(url): 
        Reponse = requests.get(url)
        soup = BeautifulSoup(Reponse.text, "html.parser")
        table = soup.find("table",{"class":"table"})
        df = []
        for valeur in table.find_all('tbody'):
            rows = table.find_all('tr')
            for row in rows:
                donnee = row.find('td', class_ ="text-center bg-primary")
                # the last row of our table (avis) don't have values (return None type error)
                # so we add a string in our list (fefekeff) to replace them.
                if donnee != None: 
                    donnee= donnee.text.strip()
                else:
                    donnee = 'fefekeff'
                df.append(donnee)
        return(df) 
        # end of the fonction for getting one url data
        
        
    # calling the get_data function for all urls
    df =  []
    for i in pages: 
        time.sleep(random.randrange(1,5))
        dataset = get_data(i)
        df.append(dataset)   
    return df
    



In [7]:
data = get_all_data(pages)


### Creating the dataframe to store extracted data

In [8]:
column = ['date','Tempmax_°C','Tempmin_°C','Vitesse_vent_km/h','Temp_vent_°C','Precipitation_mm',
           'Humidite_%','Visibilite_km','Couverture_nuageuse_%','Indice_chaleur','Point_rosee_°C',
           'Pression_hpa','lever_soleil','coucher_soleil','duree_jour','avis']
indexes   = pd.date_range(start=date_debut, end=date_fin)

In [9]:
def store_data(data,col): 
    dataset = pd.DataFrame(data,columns=col) # we will use the indexes at the end 
    return(dataset)

In [10]:
dataset = store_data(data,column)
print(dataset.shape)
dataset.head()

(16, 16)


,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
0,15/04/2022,22°,10°,13km/h,9°,1mm,96%,7.875km,60%,25,14°C,1023hPa,07:12:00,20:37:00,13:25:0,fefekeff
1,16/04/2022,23°,11°,21km/h,9°,0mm,96%,10km,20%,25,13°C,1022hPa,07:10:00,20:38:00,13:28:0,fefekeff
2,17/04/2022,22°,9°,10km/h,8°,0mm,90%,10km,1%,24,11°C,1021hPa,07:09:00,20:40:00,13:31:0,fefekeff
3,18/04/2022,18°,10°,23km/h,7°,1mm,88%,10km,35%,18,11°C,1020hPa,07:07:00,20:41:00,13:34:0,fefekeff
4,19/04/2022,13°,9°,17km/h,8°,2mm,92%,8km,80%,13,10°C,1016hPa,07:05:00,20:42:00,13:37:0,fefekeff


In [11]:
dataset[dataset.isnull().any(axis=1)]

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
11,26/04/2022,21°,8°,11km/h,7°,87%,10km,13%,21,12°C,1015hPa,06:54:00,20:50:00,13:56:0,fefekeff,None
12,27/04/2022,21°,11°,21km/h,9°,79%,10km,38%,21,12°C,1018hPa,06:53:00,20:52:00,13:59:0,fefekeff,None


In [12]:
brute = dataset.copy()
brute["ville"]= ville
# brute.to_csv("data/brute.csv")

### Correction of our dataset

* If there is no column precipitation( precipitation =0 mm), our table is shift. 
* we're going to filter shifted lines in dataframe, 
* shift them in the other sens to correct them
* Then we sort values by date and now we can set indexes.

By observing the data, we have seen that 
the shifted lines are those where we find Nan or None values.
na_ligne = pd.DataFrame(dataset[dataset.isnull().any(axis = 1)])

In [13]:
# creation of a function to clean and replace wrong data

def replace_data(dataset,col):
     # getting  the shiffeted line 
    na_ligne = pd.DataFrame(dataset[dataset.isnull().any(axis = 1)])
    
    
    #creation of a function to shif and clean data  
    def decal_data(na_ligne):
    # we'll trunc the dataset because the shift function put the none value 
    # at the first position of the list and here we want none at the Precipitations column.
        correct_val = []
        for i in range(len(na_ligne)):# Loop all na values rows
            df = na_ligne.iloc[i,:]
            df_coupe_debut = df[:'Temp_vent_°C'] # cut the line in 2 parts
            df_coupe_fin = df.loc['Precipitation_mm':] # the line to shift
            df_coupe_fin = df_coupe_fin.shift(periods=1,fill_value = 0)
            df = list(it.chain(df_coupe_debut, df_coupe_fin))# merge lines
            correct_val.append(df)
        return(correct_val)
    
    correct_val = decal_data(na_ligne)# correction of wrong lines
    correct_val = pd.DataFrame(correct_val,columns = col)
    cleaned_dataset = dataset.dropna() # Deleate wrong lines from the dataset
    cleaned_dataset = cleaned_dataset.append(correct_val)#add corrected lines in the dataset
    cleaned_dataset.reset_index()
    cleaned_dataset.sort_values(by=['date'], inplace = True)
    return(cleaned_dataset)

In [14]:
cleaned_dataset = replace_data(dataset,column)
cleaned_dataset = cleaned_dataset.set_index(indexes)
cleaned_dataset.head()

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2022-04-15,15/04/2022,22°,10°,13km/h,9°,1mm,96%,7.875km,60%,25,14°C,1023hPa,07:12:00,20:37:00,13:25:0,fefekeff
2022-04-16,16/04/2022,23°,11°,21km/h,9°,0mm,96%,10km,20%,25,13°C,1022hPa,07:10:00,20:38:00,13:28:0,fefekeff
2022-04-17,17/04/2022,22°,9°,10km/h,8°,0mm,90%,10km,1%,24,11°C,1021hPa,07:09:00,20:40:00,13:31:0,fefekeff
2022-04-18,18/04/2022,18°,10°,23km/h,7°,1mm,88%,10km,35%,18,11°C,1020hPa,07:07:00,20:41:00,13:34:0,fefekeff
2022-04-19,19/04/2022,13°,9°,17km/h,8°,2mm,92%,8km,80%,13,10°C,1016hPa,07:05:00,20:42:00,13:37:0,fefekeff


In [15]:
cleaned_dataset[cleaned_dataset.isnull().any(axis=1)]

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis


Sometimes, we need to shift twice or more the data to fix the problem.
Above we can see that Pression_hpa is at point_Rosee place. So we shift the data at that place

In [16]:
def replace_data1(dataset,col):
    na_ligne = pd.DataFrame(dataset[dataset.isnull().any(axis = 1)]) # getting  the shiffeted line 
    def decal_data(na_ligne):
    # we'll trunc the dataset because the shift function put the none value at the first position of the list and here we want none at the Precipitations column.
        correct_val = []
        for i in range(len(na_ligne)):
            df = na_ligne.iloc[i,:]
            df_coupe_debut = df[:'Couverture_nuageuse_%']
            df_coupe_fin = df.loc['Indice_chaleur':]
            df_coupe_fin = df_coupe_fin.shift(periods=1,fill_value = 0)
            df = list(it.chain(df_coupe_debut, df_coupe_fin))
            correct_val.append(df)
        return(correct_val)
    
    correct_val = decal_data(na_ligne)
    correct_val = pd.DataFrame(correct_val,columns = col)
    cleaned_dataset = dataset.dropna()
    cleaned_dataset = cleaned_dataset.append(correct_val)
    cleaned_dataset.reset_index()
    cleaned_dataset.sort_values(by=['date'], inplace = True)
    return(cleaned_dataset)

In [17]:
cleaned_dataset = replace_data1(cleaned_dataset,column)
cleaned_dataset.head()

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2022-04-15,15/04/2022,22°,10°,13km/h,9°,1mm,96%,7.875km,60%,25,14°C,1023hPa,07:12:00,20:37:00,13:25:0,fefekeff
2022-04-16,16/04/2022,23°,11°,21km/h,9°,0mm,96%,10km,20%,25,13°C,1022hPa,07:10:00,20:38:00,13:28:0,fefekeff
2022-04-17,17/04/2022,22°,9°,10km/h,8°,0mm,90%,10km,1%,24,11°C,1021hPa,07:09:00,20:40:00,13:31:0,fefekeff
2022-04-18,18/04/2022,18°,10°,23km/h,7°,1mm,88%,10km,35%,18,11°C,1020hPa,07:07:00,20:41:00,13:34:0,fefekeff
2022-04-19,19/04/2022,13°,9°,17km/h,8°,2mm,92%,8km,80%,13,10°C,1016hPa,07:05:00,20:42:00,13:37:0,fefekeff


In [18]:
cleaned_dataset[cleaned_dataset.isnull().any(axis=1)]

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis


In [19]:
cleaned_dataset["Pression_hpa"].unique()

array(['1023hPa', '1022hPa', '1021hPa', '1020hPa', '1016hPa', '1013hPa',
       '1006hPa', '1003hPa', '1004hPa', '1011hPa', '1015hPa', '1018hPa',
       '1026hPa'], dtype=object)

In [20]:
cleaned_dataset["Precipitation_mm"].unique()

array(['1mm', '0mm', '2mm', '3mm', '4mm', '5mm', 0], dtype=object)

In [21]:
cleaned_dataset["Tempmin_°C"].unique()

array(['10°', '11°', '9°', '7°', '8°', '6°', '12°', '13°'], dtype=object)

Now we'll remove caracters in our dats('° % km/h °C hPa')

In [22]:
def remove_carcter(dataset):
    for i in dataset.columns:
        dataset[i] = dataset[i].str.strip('° % km/h °C hPa')
    return dataset



In [23]:
final_dataset = remove_carcter(cleaned_dataset)
final_dataset

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2022-04-15,15/04/2022,22,10,13,9,1,96,7.875,60,25,14,1023,07:12:00,20:37:00,13:25:0,fefekeff
2022-04-16,16/04/2022,23,11,21,9,0,96,10,20,25,13,1022,07:10:00,20:38:00,13:28:0,fefekeff
2022-04-17,17/04/2022,22,9,10,8,0,90,10,1,24,11,1021,07:09:00,20:40:00,13:31:0,fefekeff
2022-04-18,18/04/2022,18,10,23,7,1,88,10,35,18,11,1020,07:07:00,20:41:00,13:34:0,fefekeff
2022-04-19,19/04/2022,13,9,17,8,2,92,8,80,13,10,1016,07:05:00,20:42:00,13:37:0,fefekeff
2022-04-20,20/04/2022,8,7,19,3,3,94,6.75,100,8,7,1013,07:04:00,20:43:00,13:39:0,fefekeff
2022-04-21,21/04/2022,12,8,13,7,4,98,6.25,100,12,9,1006,07:02:00,20:44:00,13:42:0,fefekeff
2022-04-22,22/04/2022,20,6,11,5,1,94,9.875,32,20,11,1003,07:01:00,20:46:00,13:45:0,fefekeff
2022-04-23,23/04/2022,14,8,27,6,3,90,9.75,59,14,11,1004,06:59:00,20:47:00,13:48:0,fefekeff
2022-04-24,24/04/2022,13,10,36,6,5,91,9.625,94,13,10,1011,06:57:00,20:48:00,13:51:0,fefekeff


In [24]:
final_dataset[final_dataset.isnull().any(axis=1)]

,date,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2022-04-26,26/04/2022,21,8,11,7,NaN,87,10,13,21,12,1015,06:54:00,20:50:00,13:56:0,fefekeff
2022-04-27,27/04/2022,21,11,21,9,NaN,79,10,38,21,12,1018,06:53:00,20:52:00,13:59:0,fefekeff


In [25]:
final_dataset["Precipitation_mm"].fillna(0,inplace=True)

In [26]:
final_dataset["Precipitation_mm"].unique()

array(['1', '0', '2', '3', '4', '5', 0], dtype=object)

Now that all columns are at there place we can handle dtypes of the features.

In [27]:
final_dataset.set_index("date",inplace=True)

In [28]:
# selecting columns which interest us and remove NaNs
final_dataset = final_dataset[['Tempmax_°C', 'Tempmin_°C','Vitesse_vent_km/h',
                               'Precipitation_mm', 'Humidite_%','Pression_hpa']]
final_dataset = final_dataset.fillna(0)
final_dataset.head()

,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Precipitation_mm,Humidite_%,Pression_hpa
date,,,,,,
15/04/2022,22,10,13,1,96,1023
16/04/2022,23,11,21,0,96,1022
17/04/2022,22,9,10,0,90,1021
18/04/2022,18,10,23,1,88,1020
19/04/2022,13,9,17,2,92,1016


In [29]:
def convertfloat(data):
    for i in data.columns:
        data[i] = data[i].astype(float)
    return data


In [30]:
final_dataset = convertfloat(final_dataset)
final_dataset.dtypes

Tempmax_°C           float64
Tempmin_°C           float64
Vitesse_vent_km/h    float64
Precipitation_mm     float64
Humidite_%           float64
Pression_hpa         float64
dtype: object

In [31]:
def convertint(data):
    for i in data.columns:
        data[i] = data[i].astype(int)
    return data

In [32]:
final_dataset = convertint(final_dataset)
final_dataset.dtypes

Tempmax_°C           int32
Tempmin_°C           int32
Vitesse_vent_km/h    int32
Precipitation_mm     int32
Humidite_%           int32
Pression_hpa         int32
dtype: object

In [33]:
final_dataset["ville"]= ville
final_dataset["Region"]= Region
final_dataset.head()

,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Precipitation_mm,Humidite_%,Pression_hpa,ville,Region
date,,,,,,,,
15/04/2022,22,10,13,1,96,1023,Toulouse,Occitanie
16/04/2022,23,11,21,0,96,1022,Toulouse,Occitanie
17/04/2022,22,9,10,0,90,1021,Toulouse,Occitanie
18/04/2022,18,10,23,1,88,1020,Toulouse,Occitanie
19/04/2022,13,9,17,2,92,1016,Toulouse,Occitanie


In [34]:
final_dataset = final_dataset[["Region","ville",'Tempmax_°C',
                                    'Tempmin_°C', 'Vitesse_vent_km/h',
                                    'Precipitation_mm', 'Humidite_%','Pression_hpa']]

### final_dataset

In [35]:
# final_dataset.to_csv("data/dataMeteoVille/MeteoLyon2022.csv",sep=';')
#final_dataset.to_csv("data/dataMeteoVille/MeteoParis2022.csv",sep=';')
# final_dataset.to_csv("data/dataMeteoVille/MeteoBordeaux2022.csv",sep=';')
# final_dataset.to_csv("data/dataMeteoVille/MeteoToulouse2022.csv",sep=';')
# final_dataset.to_csv("data/dataMeteoVille/MeteoMarseille2022.csv",sep=';')

In [36]:
# cleaned_dataset.to_csv("data/brutplus.csv")

In [37]:
# final_dataset.to_csv("data/dataMeteoVille/MeteoBordeauxAvril2022.csv",sep=';')
#final_dataset.to_csv("data/dataMeteoVille/MeteoParisAvril2022.csv",sep=';')
# final_dataset.to_csv("data/dataMeteoVille/MeteoLyonAvril2022.csv",sep=';')
# final_dataset.to_csv("data/dataMeteoVille/MeteoMarseilleAvril2022.csv",sep=';')
final_dataset.to_csv("data/dataMeteoVille/MeteoToulouseAvril2022.csv",sep=';')